In [1]:
import pandas as pd
import warnings
from IPython.utils import io
import sys
warnings.filterwarnings('ignore')
#pd.set_option('display.max_columns', None)

## Load filtered HCRIS PUF file

In [2]:
puf_df = pd.read_pickle('~/GitHub/HCRIS-databuilder/Filtered_PUF_data/FilteredEngineeredPUF_p5.pkl')
puf_df = puf_df.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
puf_df.columns = puf_df.columns.str.strip()
puf_df.head()

RPT_REC_NUM S2_1_C2_2  S2_1_C1_35 S2_1_C1_40 S2_1_C2_40  E_A_HOS_C1_72  \
2089      699112        TX         1.0          Y          N        76191.0   
3058      645789        OK         NaN          Y          N       327887.0   
3059      725518        TN         NaN          Y          N         5475.0   
3060      632292        VA         NaN          N          Y         4965.0   
3061      659081        VA         NaN          Y          Y         7048.0   

      E_A_HOS_C1_7096  E_A_HOS_C1_7097  E_A_HOS_C1_7093  E_A_HOS_C1_7090  ...  \
2089            874.0          14695.0              NaN              NaN  ...   
3058              NaN          72394.0              NaN              NaN  ...   
3059              NaN              NaN              NaN              NaN  ...   
3060              NaN              NaN              NaN              NaN  ...   
3061              NaN              NaN              NaN              NaN  ...   

      E_A_HOS_C1_68  E_A_HOS_C1_93  E_A_HOS_C1_47  E_A_HOS_C1_48  \
2089            NaN            NaN        57819.0         5749.0   
3058            NaN            NaN       300956.0       265783.0   
3059            NaN            NaN            NaN            NaN   
3060            NaN            NaN         6979.0            NaN   
3061            NaN            NaN         7428.0            NaN   

      E_A_HOS_C1_7099  Line 19 Subtotal  \
2089             53.0           62273.0   
3058             59.0          322008.0   
3059             63.0               NaN   
3060             75.0            7542.0   
3061             80.0            8036.0   

      Reconstructed IPPS payment (pre HAC penalty)  Reconstructed HAC penalty  \
2089                                       77792.0                      778.0   
3058                                      394349.0                     3943.0   
3059                                           0.0                        NaN   
3060                                        7542.0                       75.0   
3061                                        8036.0                       80.0   

      Reconstructed IPPS payment (post HAC penalty)  \
2089                                        77014.0   
3058                                       390406.0   
3059                                            0.0   
3060                                         7467.0   
3061                                         7956.0   

      HAC penalty imputed from E_A_HOS_C1_59  
2089                                  622.73  
3058                                 3220.08  
3059                                     NaN  
3060                                   75.42  
3061                                   80.36  

[5 rows x 26 columns]

## **Load SAS database files**

In [3]:
with io.capture_output() as captured: main_df = pd.read_sas('~/Desktop/HCRIS/hosp10-sas/prds_hosp10_yr2010.sas7bdat')
main_df['File Date'] = ['2010'] * main_df.shape[0]
main_df = main_df.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
main_df['rpt_rec_num'] = main_df['rpt_rec_num'].astype(int)
main_df['rpt_rec_num'] = main_df['rpt_rec_num'].astype(str)
main_df = main_df.replace(r"^ +| +$", r"", regex=True)
main_df.columns = main_df.columns.str.strip()
print('2010: (rows, columns) =', main_df.shape)

yrs = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
for yr in yrs:
    with io.capture_output() as captured: tdf = pd.read_sas('~/Desktop/HCRIS/hosp10-sas/prds_hosp10_yr' + yr + '.sas7bdat')
    tdf['File Date'] = [yr] * tdf.shape[0]
    tdf = tdf.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
    tdf['rpt_rec_num'] = tdf['rpt_rec_num'].astype(int)
    tdf['rpt_rec_num'] = tdf['rpt_rec_num'].astype(str)
    tdf = tdf.replace(r"^ +| +$", r"", regex=True)
    tdf.columns = tdf.columns.str.strip()
    main_df = pd.concat([main_df, tdf], ignore_index=True)
    print(yr + ': (rows, columns) =', main_df.shape)

del tdf

2010: (rows, columns) = (2323, 5522)
2011: (rows, columns) = (8473, 5522)
2012: (rows, columns) = (14700, 5522)
2013: (rows, columns) = (20948, 5522)
2014: (rows, columns) = (27198, 5522)
2015: (rows, columns) = (33455, 5522)
2016: (rows, columns) = (39666, 5522)
2017: (rows, columns) = (45840, 5522)
2018: (rows, columns) = (52000, 5522)
2019: (rows, columns) = (58121, 5522)
2020: (rows, columns) = (64178, 5522)
2021: (rows, columns) = (67886, 5522)
2022: (rows, columns) = (67904, 5522)


In [4]:
to_replace = ['rpt_rec_num', 'prvdr_num', 'fi_num', 'rpt_stus_cd', 'fi_creat_dt', 
              'fy_bgn_dt', 'fy_end_dt', 'util_cd', 'trnsmtl_num', 'state', 
              'st_cty_cd', 'census', 'region', 'proc_dt', 'msa', 'H3_1_HHA1_C10_15____',
              'H3_1_HHA1_C11_15____', 'sub3', 
             ]

replacement = ['RPT_REC_NUM', 'PRVDR_NUM', 'FI_NUM', 'RPT_STUS_CD', 'FI_CREAT_DT', 
               'FY_BGN_DT', 'FY_END_DT', 'UTIL_CODE', 'TRNSMTL_NUM', 'STATE', 
               'ST_CTY_CD', 'CENSUS', 'REGION', 'PROC_DT', 'MSA', 'H3_1_HHA1_C10_15',
               'H3_1_HHA1_C11_15', 'SUB3',
              ]

main_df.rename(columns = {to_replace[0]: replacement[0], to_replace[1]: replacement[1],
                           to_replace[2]: replacement[2], to_replace[3]: replacement[3],
                           to_replace[4]: replacement[4], to_replace[5]: replacement[5],
                           to_replace[6]: replacement[6], to_replace[7]: replacement[7],
                           to_replace[8]: replacement[8], to_replace[9]: replacement[9],
                           to_replace[10]: replacement[10], to_replace[11]: replacement[11],
                           to_replace[12]: replacement[12], to_replace[13]: replacement[13],
                           to_replace[14]: replacement[14], to_replace[15]: replacement[15],
                           to_replace[16]: replacement[16], to_replace[17]: replacement[17],
                         }, inplace = True)

main_df.drop(labels=['_NAME_', 'E_A_HOS_C1_68', 'E_A_HOS_C1_7090', 'E_A_HOS_C1_7091', 'E_A_HOS_C1_7093', 
                     'E_A_HOS_C1_7096', 'E_A_HOS_C1_93', 'E_A_HOS_C1_47', 'E_A_HOS_C1_49', 'E_A_HOS_C1_50', 
                     'E_A_HOS_C1_54', 'E_A_HOS_C1_59', 'E_A_HOS_C1_7094', 'E_A_HOS_C1_72', 'E_A_HOS_C1_7099', 
                     'E_A_HOS_C1_7097', 'E_A_HOS_C1_48', 'S2_1_C1_35', 'S2_1_C2_2'], axis=1, inplace=True)

print('main_df.shape:', main_df.shape)

main_df.shape: (67904, 5503)


In [5]:
common_features = list(filter(lambda x:x in list(puf_df), list(main_df)))
print(common_features)

main_df['RPT_REC_NUM'] = main_df['RPT_REC_NUM'].astype(int)
main_df['RPT_REC_NUM'] = main_df['RPT_REC_NUM'].astype(str)
main_df = main_df.merge(puf_df, how='outer', on=common_features)
del puf_df

main_df.sort_values(by='Reconstructed HAC penalty', inplace=True, ascending=False)

print(main_df.shape)
main_df.head()

['RPT_REC_NUM']
(67904, 5528)


RPT_REC_NUM PRVDR_NUM FI_NUM RPT_STUS_CD FI_CREAT_DT  FY_BGN_DT  \
57096      716685    330101  13001           5  2022-04-27 2019-01-01   
59857      696215    330101  13001           1  2021-08-11 2020-01-01   
48734      689516    330101  13001           5  2021-04-09 2018-01-01   
62318      711561    330214  13001           1  2022-02-08 2020-09-01   
44615      707313    330101  13001           2  2021-12-10 2017-01-01   

       FY_END_DT UTIL_CODE TRNSMTL_NUM STATE  ... E_A_HOS_C1_68  \
57096 2019-12-31         F           N    33  ...      199410.0   
59857 2020-12-31         F           N    33  ...           NaN   
48734 2018-12-31         F           N    33  ...           NaN   
62318 2021-08-31         F           N    33  ...           NaN   
44615 2017-12-31         F           M    33  ...         394.0   

       E_A_HOS_C1_93  E_A_HOS_C1_47  E_A_HOS_C1_48 E_A_HOS_C1_7099  \
57096            NaN    738759903.0            NaN       2196646.0   
59857            NaN    659790459.0            NaN       7823406.0   
48734            NaN    671153761.0            NaN             NaN   
62318            NaN    627815592.0            NaN             NaN   
44615            NaN    638773907.0            NaN             NaN   

      Line 19 Subtotal  Reconstructed IPPS payment (pre HAC penalty)  \
57096      859459636.0                                   856986882.0   
59857      784623441.0                                   782340652.0   
48734      777272737.0                                   777557401.0   
62318      763550708.0                                   765197446.0   
44615      730275546.0                                   731160850.0   

       Reconstructed HAC penalty  \
57096                  8569869.0   
59857                  7823407.0   
48734                  7775574.0   
62318                  7651974.0   
44615                  7311608.0   

       Reconstructed IPPS payment (post HAC penalty)  \
57096                                    848417013.0   
59857                                    774517245.0   
48734                                    769781827.0   
62318                                    757545472.0   
44615                                    723849242.0   

       HAC penalty imputed from E_A_HOS_C1_59  
57096                              9385443.34  
59857                              8618696.18  
48734                              8481422.96  
62318                              8496772.62  
44615                              7977062.69  

[5 rows x 5528 columns]

In [6]:
crosswalk_df = pd.read_csv('~/GitHub/HCRIS-databuilder/crosswalk/2552-10 SAS FILE RECORD LAYOUT AND CROSSWALK TO 96 - 2021.csv', sep=',')
crosswalk_labels = crosswalk_df['10_FIELD_NAME'].tolist()
crosswalk_labels = [str(x).strip(' ') for x in crosswalk_labels]
crosswalk_df['10_FIELD_NAME'] = crosswalk_labels

crosswalk_df = crosswalk_df.replace(r"^ +| +$", r"", regex=True)
crosswalk_df.columns = crosswalk_df.columns.str.strip()


In [10]:
main_df_col_labels = list(main_df)
main_df_col_labels = [str(x).strip(' ') for x in main_df_col_labels]

print('\nNumber of labels in the crosswalk:', len(crosswalk_labels))
print('Number of unique labels in the crosswalk:', len(list(set(crosswalk_labels))))

print('\nNumber of labels in the main_df:', len(main_df_col_labels))
print('Number of unique labels in the main_df:', len(list(set(main_df_col_labels))))

shared_labels = list(set(main_df_col_labels) & set(crosswalk_labels))
print('\nNumber of labels shared between the main dataframe and the crosswalk:', len(shared_labels))
shared_labels.append('File Date')

dif = set(main_df_col_labels).difference(crosswalk_labels)
print('\n' + str(len(dif)) + ' labels in main dataframe but not in crosswalk:')
print(dif)

dif = set(crosswalk_labels).difference(main_df_col_labels)
print('\n' + str(len(dif)) + ' labels in crosswalk but not in main dataframe:')
print(dif)

del main_df_col_labels
del dif


Number of labels in the crosswalk: 5573
Number of unique labels in the crosswalk: 5518

Number of labels in the main_df: 3890
Number of unique labels in the main_df: 3890

Number of labels shared between the main dataframe and the crosswalk: 3884

6 labels in main dataframe but not in crosswalk:
{'Reconstructed IPPS payment (pre HAC penalty)', 'File Date', 'Reconstructed IPPS payment (post HAC penalty)', 'Line 19 Subtotal', 'Reconstructed HAC penalty', 'HAC penalty imputed from E_A_HOS_C1_59'}

1634 labels in crosswalk but not in main dataframe:
{'D_5_HOS_C6_89', 'S2_1_C2_134', 'D_4_SUB1_C13_67', 'H4_2_HHA3_C2_27', 'H4_2_HHA5_C2_35', 'H4_2_HHA5_C2_16', 'E_B_SUB1_C1_18', 'E_A_HOS_C1_2899', 'D3_SUB1_C3_73', 'D3_SNF_C2_46', 'E3_4_SUB1_C1_6', 'D_4_IRF_C11_89', 'H3_1_HHA4_C6_2', 'D3_SUB1_C3_60', 'H4_2_HHA5_C1_29', 'D3_SNF_C3_44', 'C_1_C7_41', 'E_B_SUB2_C1_38', 'H4_2_HHA4_C2_10', 'D_4_SUB2_C11_69', 'S4_HHA5_C5_36', 'D3_IRF_C2_89', 'D_4_HOS_C12_88', 'H4_2_HHA4_C2_19', 'E_B_SUB2_C1_23', 'D_2_

In [11]:
add_labels = ['Line 19 Subtotal', 
              'Reconstructed HAC penalty', 
              'HAC penalty imputed from E_A_HOS_C1_59',
              'Reconstructed IPPS payment (post HAC penalty)', 
              'Reconstructed IPPS payment (pre HAC penalty)',
              ]

main_df = main_df[main_df.columns[main_df.columns.isin(shared_labels + add_labels)]]
del shared_labels

main_df = main_df.dropna(axis=1, how='all')
main_df = main_df.dropna(axis=0, how='all')
print('main_df.shape (columns and rows with no data removed):', main_df.shape)

CODE = []
FIELD_DESCRIPTION = []
TYPE = []
SUBTYPE = []

col_labels = list(main_df)

for lab in col_labels:
    if lab == 'File Date':
        CODE.append('File Date')
        FIELD_DESCRIPTION.append('File Date')
        TYPE.append('File Date')
        SUBTYPE.append('File Date')
        
        
    elif lab in add_labels:
        CODE.append('')
        FIELD_DESCRIPTION.append(lab)
        TYPE.append('CALCULATION OF REIMBURSEMENT SETTLEMENT (PPS)')
        SUBTYPE.append('')
        
    else:
        df_sub = crosswalk_df[crosswalk_df['10_FIELD_NAME'] == lab]
        CODE.append(df_sub['10_FIELD_NAME'].iloc[0])
        
        x = df_sub['FIELD DESCRIPTION'].iloc[0]
        if x == "" or pd.isnull(x):
            FIELD_DESCRIPTION.append('No Description')
            
        else:
            FIELD_DESCRIPTION.append(x)
        
        x = df_sub['TYPE'].iloc[0]
        if x == "" or pd.isnull(x):
            TYPE.append('No Description')
        else:
            TYPE.append(x)
        
        x = df_sub['SUBTYPE'].iloc[0]
        if x == "" or pd.isnull(x):
                SUBTYPE.append('')
        else:
            SUBTYPE.append(x)

SUBTYPE = pd.Series(SUBTYPE).fillna('').tolist()
TYPE = pd.Series(TYPE).fillna('').tolist()

for i, val in enumerate(SUBTYPE):
    if val == 'File Date':
        continue
    elif val == '' and CODE[i] == '':
        SUBTYPE[i] = str(FIELD_DESCRIPTION[i])
    elif val == '':
        SUBTYPE[i] = str(FIELD_DESCRIPTION[i]) + ' ' + '(' + str(CODE[i]) + ')'
    else:
        SUBTYPE[i] = str(val) + ' ' + str(FIELD_DESCRIPTION[i]) + ' (' + str(CODE[i]) + ')'


main_df.shape (columns and rows with no data removed): (67904, 3890)


In [12]:
df2 = pd.DataFrame([col_labels, FIELD_DESCRIPTION, TYPE, SUBTYPE], columns=col_labels)
main_df = pd.concat([df2, main_df])
del df2

main_df.columns = pd.MultiIndex.from_arrays(main_df.iloc[0:4].values)
main_df = main_df.iloc[4:]

del col_labels
del FIELD_DESCRIPTION
del TYPE
del SUBTYPE

#main_df.head()


In [13]:
CMS_Gen_Info_df = pd.read_csv('~/GitHub/HCRIS-databuilder/GeoData/Hospital_General_Information.tsv', sep='\t')
print(list(CMS_Gen_Info_df))
CMS_Gen_Info_df.head()

['Facility ID', 'Facility Name', 'Address', 'City', 'State', 'ZIP Code', 'County Name', 'Phone Number', 'Hospital Type', 'Hospital Ownership', 'Emergency Services', 'Meets criteria for promoting interoperability of EHRs', 'Hospital overall rating', 'Hospital overall rating footnote', 'Mortality national comparison', 'Mortality national comparison footnote', 'Safety of care national comparison', 'Safety of care national comparison footnote', 'Readmission national comparison', 'Readmission national comparison footnote', 'Patient experience national comparison', 'Patient experience national comparison footnote', 'Effectiveness of care national comparison', 'Effectiveness of care national comparison footnote', 'Timeliness of care national comparison', 'Timeliness of care national comparison footnote', 'Efficient use of medical imaging national comparison', 'Efficient use of medical imaging national comparison footnote', 'Location']


Facility ID                              Facility Name  \
0      030084  CHINLE COMPREHENSIVE HEALTH CARE FACILITY   
1      044022                   CONWAY BEHAVIORAL HEALTH   
2      10021F             96th Medical Group (Eglin AFB)   
3      170204                ROCK REGIONAL HOSPITAL, LLC   
4      171322                   HAMILTON COUNTY HOSPITAL   

                           Address       City State  ZIP Code County Name  \
0        US HWY 191, HOSPITAL ROAD     CHINLE    AZ     86503      APACHE   
1                2255 STURGIS ROAD     CONWAY    AR     72034    FAULKNER   
2  96 MDG 307 Boatner Rd Suite 114  Eglin AFB    FL     32542    OKALOOSA   
3             3251 NORTH ROCK ROAD      DERBY    KS     67037    SEDGWICK   
4                  700 NORTH HUSER   SYRACUSE    KS     67878    HAMILTON   

     Phone Number                       Hospital Type     Hospital Ownership  \
0  (928) 674-7001                Acute Care Hospitals   Government - Federal   
1  (501) 205-0011                         Psychiatric            Proprietary   
2  (850) 883-8600  Acute Care - Department of Defense  Department of Defense   
3  (833) 345-7625                Acute Care Hospitals            Proprietary   
4  (620) 384-7461           Critical Access Hospitals     Government - Local   

   ...  Readmission national comparison footnote  \
0  ...                                       NaN   
1  ...                                      19.0   
2  ...                                      22.0   
3  ...                                       5.0   
4  ...                                      16.0   

  Patient experience national comparison  \
0           Same as the national average   
1                          Not Available   
2                          Not Available   
3                          Not Available   
4                          Not Available   

  Patient experience national comparison footnote  \
0                                             NaN   
1                                            19.0   
2                                            22.0   
3                                             5.0   
4                                            16.0   

   Effectiveness of care national comparison  \
0                 Below the national average   
1                              Not Available   
2                              Not Available   
3                              Not Available   
4                              Not Available   

  Effectiveness of care national comparison footnote  \
0                                                NaN   
1                                               19.0   
2                                               22.0   
3                                                5.0   
4                                                5.0   

   Timeliness of care national comparison  \
0                           Not Available   
1                           Not Available   
2                           Not Available   
3                           Not Available   
4            Same as the national average   

  Timeliness of care national comparison footnote  \
0                                             5.0   
1                                            19.0   
2                                            22.0   
3                                             5.0   
4                                             NaN   

   Efficient use of medical imaging national comparison  \
0                                      Not Available      
1                                      Not Available      
2                                      Not Available      
3                                      Not Available      
4                                      Not Available      

  Efficient use of medical imaging national comparison footnote  \
0                                               16.0              
1                                               19.0              
2                           

In [14]:
hospital_ls = main_df.iloc[:, (main_df.columns.get_level_values(0) == 'S2_1_C1_3')].T.values.tolist()[0]
id_ls = main_df.iloc[:, (main_df.columns.get_level_values(0) == 'PRVDR_NUM')].T.values.tolist()[0]

print(hospital_ls[0:4])
print(id_ls[0:4])

['NEW YORK PRESBYTERIAN HOSPITAL', 'NEW YORK PRESBYTERIAN HOSPITAL', 'NEW YORK PRESBYTERIAN HOSPITAL', 'NYU LANGONE HOSPITALS']
['330101', '330101', '330101', '330214']


In [15]:
lats = []
lons = []
Htypes = []
Ctypes = []

misses = 0
hits = 0
for i, h in enumerate(hospital_ls):
    
    fid = id_ls[i]
    
    try:
        df = CMS_Gen_Info_df[CMS_Gen_Info_df['Facility ID'] == fid]
            
        loc = df['Location'].iloc[0]
        loc = loc.replace("POINT (","") 
        loc = loc.replace(")","")
        loc = loc.split(" ")
            
        lat = loc[1]
        lon = loc[0]
        lats.append(lat)
        lons.append(lon)
        
        htype = df['Hospital Type'].iloc[0]
        Htypes.append(htype)
        ctype = df['Hospital Ownership'].iloc[0]
        Ctypes.append(ctype)
        
        del df
        
    except:
        lats.append(float('NaN'))
        lons.append(float('NaN'))
        Htypes.append(float('NaN'))
        Ctypes.append(float('NaN'))


In [16]:
for i, val in enumerate(lats):
    if pd.isnull(val): 
        continue
    else:
        print(val, lons[i], Htypes[i], Ctypes[i])
    if i > 10:
        break

40.76415 -73.954335 Acute Care Hospitals Voluntary non-profit - Private
40.76415 -73.954335 Acute Care Hospitals Voluntary non-profit - Private
40.76415 -73.954335 Acute Care Hospitals Voluntary non-profit - Private
40.74226 -73.974661 Acute Care Hospitals Voluntary non-profit - Private
40.76415 -73.954335 Acute Care Hospitals Voluntary non-profit - Private
40.76415 -73.954335 Acute Care Hospitals Voluntary non-profit - Private
40.76415 -73.954335 Acute Care Hospitals Voluntary non-profit - Private
40.74226 -73.974661 Acute Care Hospitals Voluntary non-profit - Private
34.075496 -118.380122 Acute Care Hospitals Voluntary non-profit - Other
34.075496 -118.380122 Acute Care Hospitals Voluntary non-profit - Other
34.075496 -118.380122 Acute Care Hospitals Voluntary non-profit - Other
34.075496 -118.380122 Acute Care Hospitals Voluntary non-profit - Other


In [17]:
print(list(main_df))

[('RPT_REC_NUM', 'Report Record Number', 'HOSPITAL IDENTIFICATION INFORMATION', 'Report Record Number (RPT_REC_NUM)'), ('PRVDR_NUM', 'Hospital Provider Number', 'HOSPITAL IDENTIFICATION INFORMATION', 'Hospital Provider Number (PRVDR_NUM)'), ('FI_NUM', 'Fiscal Intermediary Number', 'HOSPITAL IDENTIFICATION INFORMATION', 'Fiscal Intermediary Number (FI_NUM)'), ('RPT_STUS_CD', 'Report Status Code', 'HOSPITAL IDENTIFICATION INFORMATION', 'Report Status Code (RPT_STUS_CD)'), ('FI_CREAT_DT', 'Fiscal Intermediary/MAC Create Date', 'HOSPITAL IDENTIFICATION INFORMATION', 'Fiscal Intermediary/MAC Create Date (FI_CREAT_DT)'), ('FY_BGN_DT', 'Fiscal Year Begin Date', 'HOSPITAL IDENTIFICATION INFORMATION', 'Fiscal Year Begin Date (FY_BGN_DT)'), ('FY_END_DT', 'Fiscal Year End Date', 'HOSPITAL IDENTIFICATION INFORMATION', 'Fiscal Year End Date (FY_END_DT)'), ('UTIL_CODE', 'Utilization Code (L/N/F)', 'HOSPITAL IDENTIFICATION INFORMATION', 'Utilization Code (L/N/F) (UTIL_CODE)'), ('TRNSMTL_NUM', 'NOT IN

In [18]:
main_df[('Lat', 'Lat', 'Lat', 'Lat')] = lats
main_df[('Lon', 'Lon', 'Lon', 'Lon')] = lons
main_df[('Control type, text', 'Control type, text', 'Control type, text', 'Control type, text')] = Ctypes
main_df[('Hospital type, text', 'Hospital type, text', 'Hospital type, text', 'Hospital type, text')] = Htypes
main_df[('Num and Name', 'Num and Name', 'Num and Name', 'Num and Name')] = main_df[('S2_1_C1_3', 'Hospital Name', 'No Description', 'Hospital Name (S2_1_C1_3)')].astype(str) +' (' + main_df[('PRVDR_NUM', 'Hospital Provider Number', 'HOSPITAL IDENTIFICATION INFORMATION', 'Hospital Provider Number (PRVDR_NUM)')].astype(str) + ')'   

del lats
del lons
del Ctypes
del Htypes


In [19]:
badnames = ['20994', float('NaN'), '-0007', '1', '330354', '4499 ACUSHNET AVENUE OPERATING COMPA',
           '4499 ACUSHNET AVENUE OPERATING COMPM']

main_df = main_df[~main_df[('S2_1_C1_3', 'Hospital Name', 'No Description', 'Hospital Name (S2_1_C1_3)')].isin(badnames)]


In [20]:
tdf = main_df.filter(items=[('Lat', 'Lat', 'Lat', 'Lat'),
                             ('Lon', 'Lon', 'Lon', 'Lon'),
                             ('Control type, text', 'Control type, text', 'Control type, text', 'Control type, text'),
                             ('Hospital type, text', 'Hospital type, text', 'Hospital type, text', 'Hospital type, text'),
                             ('Num and Name', 'Num and Name', 'Num and Name', 'Num and Name'),
                             ('S3_1_C2_27', 'Total Facility', 'NUMBER OF BEDS', 'Total Facility (S3_1_C2_27)'),
                             ('S2_1_C2_2', 'Hospital State', 'No Description', 'Hospital State (S2_1_C2_2)'),
                            ], axis=1)

tdf.to_pickle('GenDat4App/GenDat4App_p4.pkl', protocol=4)

del tdf

In [22]:
main_df[('FY_END_DT', 'Fiscal Year End Date', 'HOSPITAL IDENTIFICATION INFORMATION', 'Fiscal Year End Date (FY_END_DT)')] = pd.to_datetime(main_df[('FY_END_DT', 'Fiscal Year End Date', 'HOSPITAL IDENTIFICATION INFORMATION', 'Fiscal Year End Date (FY_END_DT)')])
main_df = main_df.sort_values(by=[('Num and Name', 'Num and Name', 'Num and Name', 'Num and Name'),
                     ('FY_END_DT', 'Fiscal Year End Date', 'HOSPITAL IDENTIFICATION INFORMATION', 'Fiscal Year End Date (FY_END_DT)')],
                     ascending=[True, True])

In [23]:
main_df.to_pickle('hcris_all_data/HCRIS_p4.pkl', protocol=4)

print(main_df.shape[1], 'features')
print(len(main_df[('PRVDR_NUM', 
     'Hospital Provider Number', 
     'HOSPITAL IDENTIFICATION INFORMATION', 
     'Hospital Provider Number (PRVDR_NUM)')].unique()), 'hospitals')

3895 features
6858 hospitals


In [24]:

report_categories = list(set(main_df.columns.get_level_values(2).tolist()))
report_categories = [x for x in report_categories if str(x) != 'nan']
report_categories = [x for x in report_categories if str(x) != 'Lat']
report_categories = [x for x in report_categories if str(x) != 'Lon']
report_categories = [x for x in report_categories if str(x) != 'Num and Name']
report_categories = [x for x in report_categories if str(x) != 'Hospital type, text']
#report_categories = [x for x in report_categories if str(x) != '']
report_categories = [x for x in report_categories if str(x) != 'HOSPITAL IDENTIFICATION INFORMATION']

report_categories = [x for x in report_categories if str(x) != 'Control type, text']
report_categories = [x for x in report_categories if str(x) != 'HOSPITAL IDENTIFICATION INFORMATION']
report_categories.sort()

import csv

with open('GenDat4App/report_categories.csv', 'w+', newline='') as OUT:
    writer = csv.writer(OUT)
    writer.writerow(report_categories)
    
    
sub_categories = list(set(main_df.columns.get_level_values(3).tolist()))
sub_categories.sort()

with open('GenDat4App/sub_categories.csv', 'w+', newline='') as OUT:
    writer = csv.writer(OUT)
    writer.writerow(sub_categories)


In [25]:
import re

numNname = sorted(list(set(main_df[('Num and Name', 'Num and Name', 'Num and Name', 'Num and Name')].tolist())))

for i, val in enumerate(numNname):
    prvdr = re.sub('\ |\?|\.|\!|\/|\;|\:', '', val)

    tdf = main_df[main_df[('Num and Name', 'Num and Name', 'Num and Name', 'Num and Name')] == val]
    tdf.to_csv('provider_data/' + prvdr + '.csv')
    
    main_df = main_df[main_df[('Num and Name', 'Num and Name', 'Num and Name', 'Num and Name')] != val]
    
    del tdf
    
del main_df